## GRAVITY COMPENSATION

In [4]:
import pandas as pd

# Beispiel: Lesen der Daten aus einer Datei
# Pfad zur Datei
file_path = "data_1.txt"
# "C:\Users\bernh\Downloads\drive-download-20240908T150007Z-001\WitMotion New Software\WitMotion(V2024.8.12.4)\Record\2024-09-15\05-11-18-527\data_0.txt"
# Daten in den DataFrame laden, mit Tabulator als Trennzeichen
# df = pd.read_csv(file_path,'r', encoding='utf-8', sep='\t', index_col=False)
# Datei öffnen und lesen
df = pd.read_csv(file_path, sep='\t', index_col=False)

# Überprüfen der Spaltennamen nach dem Zurücksetzen des Index
# print("Spaltennamen nach Index-Zurücksetzung:", df.columns)

# Die ersten paar Zeilen nach der Anpassung anzeigen
# print(df.head(2))

#_# Ersetzen von Kommas durch Punkte in spezifischen Spalten
#_df['Chip Time()'] = df['Chip Time()'].str.replace(',', '.').astype(float)
#_# ACCELERATION:
df['Acceleration X(g)'] = df['Acceleration X(g)'].str.replace(',', '.').astype(float)
df['Acceleration Y(g)'] = df['Acceleration Y(g)'].str.replace(',', '.').astype(float)
df['Acceleration Z(g)'] = df['Acceleration Z(g)'].str.replace(',', '.').astype(float)
#_# ANGELS
df['Angle X(°)'] = df['Angle X(°)'].str.replace(',', '.').astype(float)
df['Angle Y(°)'] = df['Angle Y(°)'].str.replace(',', '.').astype(float)
df['Angle Z(°)'] = df['Angle Z(°)'].str.replace(',', '.').astype(float)
# ANGULAR VELOSITY
df['Angular velocity X(°/s)'] = df['Angular velocity X(°/s)'].str.replace(',', '.').astype(float)
df['Angular velocity Y(°/s)'] = df['Angular velocity Y(°/s)'].str.replace(',', '.').astype(float)
df['Angular velocity Z(°/s)'] = df['Angular velocity Z(°/s)'].str.replace(',', '.').astype(float)
df.head(2)

,Time,Device name,Chip Time(),Acceleration X(g),Acceleration Y(g),Acceleration Z(g),Angular velocity X(°/s),Angular velocity Y(°/s),Angular velocity Z(°/s),Angle X(°),...,Angle of course(°),GPS height(m),Number of satellites(),Position location(),Horizontal position(),Vertical position(),Quaternions 0(),Quaternions 1(),Quaternions 2(),Quaternions 3()
0,15:32:07.831,COM12,NaN,-0.095,0.013,1.005,0.0,0.0,0.0,0.577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15:32:07.929,COM12,NaN,-0.095,0.013,1.004,0.0,0.0,0.0,0.577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Berechne die Summe der Beschleunigungen (X, Y, Z) für jede Zeile der ersten 20 Zeilen
df['Sum Acceleration (g)'] = df[['Acceleration X(g)', 'Acceleration Y(g)', 'Acceleration Z(g)']].sum(axis=1)

# Optional: Nur die ersten 20 Zeilen anzeigen
df_first_20 = df.head(20)

# Ausgabe der ersten 20 Zeilen mit der Summenspalte
print(df_first_20['Sum Acceleration (g)'])

0     0.923
1     0.922
2     0.923
3     0.922
4     0.927
5     0.921
6     0.926
7     0.879
8     0.856
9     0.830
10    0.757
11    0.663
12    0.583
13    0.525
14    0.427
15    0.269
16    0.160
17    0.138
18    0.058
19   -0.104
Name: Sum Acceleration (g), dtype: float64


## KALIBRIERE DEN SENSOR
https://m.media-amazon.com/images/I/B1zDYGUvUhS.pdf

Seite 14

In [6]:
import numpy as np
import pandas as pd

# Beispiel: Ein bestehender DataFrame mit den spezifizierten Spaltennamen
data = {
    'Angle X(°)': np.linspace(0, 90, 10),  # Beispielwerte für Pitch (Angle X) in Grad
    'Angle Y(°)': np.linspace(0, 90, 10),  # Beispielwerte für Roll (Angle Y) in Grad
    'Angle Z(°)': np.linspace(0, 360, 10), # Beispielwerte für Yaw (Angle Z) in Grad
    'Acceleration X(g)': np.random.uniform(-1, 1, 10),  # Zufällige Beschleunigungswerte in g
    'Acceleration Y(g)': np.random.uniform(-1, 1, 10),
    'Acceleration Z(g)': np.random.uniform(-1, 1, 10)
}

# DataFrame erstellen
# df = pd.DataFrame(data)

# Konstante für die Erdbeschleunigung (9.81 m/s²)
g = 9.81

# Berechne die Schwerkraftkomponente (Gravitationsbeschleunigung) in Abhängigkeit von Pitch, Roll und Yaw
def compute_gravity_components(pitch_deg, roll_deg, yaw_deg):
    # Konvertiere Winkel von Grad in Bogenmaß
    pitch = np.radians(pitch_deg)
    roll = np.radians(roll_deg)
    yaw = np.radians(yaw_deg)

    # Berechne die Rotationsmatrizen für Pitch, Roll und Yaw
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(pitch), -np.sin(pitch)],
        [0, np.sin(pitch), np.cos(pitch)]
    ])

    R_y = np.array([
        [np.cos(roll), 0, np.sin(roll)],
        [0, 1, 0],
        [-np.sin(roll), 0, np.cos(roll)]
    ])

    R_z = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])

    # Gesamte Rotationsmatrix
    R = R_z @ R_y @ R_x

    # Erdbeschleunigungsvektor (im Ruhezustand nur in Z-Richtung)
    gravity = np.array([0, 0, g])

    # Berechne die Schwerkraftkomponente in den neuen Achsen
    gravity_components = R @ gravity
    return gravity_components

# Berechne für jede Zeile die Schwerkraftkomponente und subtrahiere sie von den gemessenen Beschleunigungen
gravity_corrected_acc = []
for index, row in df.iterrows():
    pitch, roll, yaw = row['Angle X(°)'], row['Angle Y(°)'], row['Angle Z(°)']
    acc_x, acc_y, acc_z = row['Acceleration X(g)'] * g, row['Acceleration Y(g)'] * g, row['Acceleration Z(g)'] * g

    # Berechne die Schwerkraftkomponenten
    gravity_components = compute_gravity_components(pitch, roll, yaw)

    # Subtrahiere die Schwerkraft von den gemessenen Beschleunigungen
    corrected_acc = np.array([acc_x, acc_y, acc_z]) - gravity_components

    # Speichere die korrigierten Beschleunigungswerte
    gravity_corrected_acc.append(corrected_acc)

# Füge die korrigierten Beschleunigungswerte zum DataFrame hinzu
df[['Corrected_Acc_X(m/s²)', 'Corrected_Acc_Y(m/s²)', 'Corrected_Acc_Z(m/s²)']] = gravity_corrected_acc

# Ergebnis anzeigen
print(df)

# Speichern des DataFrames als CSV-Datei mit Komma als Trennzeichen
csv_filename = 'imu_data_comma_new.csv'
# df.to_csv(csv_filename, index=False, sep=',')
df.to_csv(csv_filename, index=False, decimal=',')


              Time Device name  Chip Time()  Acceleration X(g)  \
0     15:32:07.831       COM12          NaN             -0.095   
1     15:32:07.929       COM12          NaN             -0.095   
2     15:32:08.027       COM12          NaN             -0.094   
3     15:32:08.127       COM12          NaN             -0.095   
4     15:32:08.227       COM12          NaN             -0.091   
..             ...         ...          ...                ...   
194   15:32:27.220       COM12          NaN             -0.023   
195   15:32:27.320       COM12          NaN             -0.023   
196   15:32:27.420       COM12          NaN             -0.023   
197   15:32:27.519       COM12          NaN             -0.023   
198   15:32:27.619       COM12          NaN             -0.023   

     Acceleration Y(g)  Acceleration Z(g)  Angular velocity X(°/s)  \
0                0.013              1.005                      0.0   
1                0.013              1.004                      0.0 